In [1]:
# Setup Notebook to load Django code
# From project root, run: jupyter-lab

import os
import sys

os.chdir('../..')

import django
django.setup()

In [22]:
import datetime as dt
import pytz

from django.utils import timezone
from django.test import override_settings

from nc.models import Stop, Person, Agency

In [13]:
durham = Agency.objects.get(pk=80)

2021-02-11 14:07:54,148 caching              DEBUG    cache hit: 7c97264763d28c00510b2512843c6df1


# Raw Stops from Stop.csv

In [12]:
# 7/31, 8:26p ET
! grep 26280425 ncdata/Stop.csv

26280425,Durham Police Department,2020-07-31 20:26:00.000,6,3,0,0,0,0,0,0,0,1168,32,


In [11]:
# 8/1, 2:25a ET
! grep 26351840 ncdata/Stop.csv

26351840,Durham Police Department,2020-08-01 02:25:00.000,5,1,0,0,0,0,0,0,0,1260,32,


# UTC date filter

In [15]:
utc = pytz.timezone("UTC")
start_date_utc = utc.localize(dt.datetime(2020, 8, 1))
start_date_utc

datetime.datetime(2020, 8, 1, 0, 0, tzinfo=<UTC>)

In [18]:
stop_utc = Stop.objects.no_cache().filter(agency=durham, date__gt=start_date_utc).order_by("date").first()
stop_utc.stop_id, stop_utc.date

(26280425, datetime.datetime(2020, 8, 1, 0, 26, tzinfo=<UTC>))

# America/New_York date filter

In [31]:
nyc = pytz.timezone("America/New_York")
start_date_nyc = nyc.localize(dt.datetime(2020, 8, 1))

In [32]:
stop_nyc = Stop.objects.no_cache().filter(agency=durham, date__gt=start_date_nyc).order_by("date").first()
stop_nyc.stop_id, stop_nyc.date

(26351840, datetime.datetime(2020, 8, 1, 6, 25, tzinfo=<UTC>))

# Django's localtime and TIME_ZONE

In [29]:
with override_settings(TIME_ZONE='UTC'):
    date = timezone.localtime(stop_utc.date)
date

datetime.datetime(2020, 8, 1, 0, 26, tzinfo=<UTC>)

In [30]:
with override_settings(TIME_ZONE="America/New_York"):
    date = timezone.localtime(stop_utc.date)
date

datetime.datetime(2020, 7, 31, 20, 26, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>)